# KNMI Weather Data Processing

This notebook processes meteorological data from the Royal Netherlands Meteorological Institute (KNMI) to support the Gelderse Poort rewilding and NDVI productivity research project.

In [ ]:
# import the necessary packages
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime

# setting of directory
output_dir = 'weatherdata_postprocessing'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

## 1. Read Initial Data

In [ ]:
raw_data_path = 'weatherdata_raw/weather.csv'

raw_data = pd.read_csv(raw_data_path, comment='#',  sep=',', header=0)

print(raw_data.head())

## 2. Data Cleaning and Transformation

In [ ]:
# transform of the date       
raw_data['date'] = pd.to_datetime(raw_data['YYYYMMDD'], format='%Y%m%d')

# transform of the unit and rename the column
processed_data = pd.DataFrame()
processed_data['date'] = raw_data['date']
processed_data['station_id'] = raw_data['STN']

# add the station name (according to the station id)
station_names = {275: 'Deelen', 391: 'Arcen'}
processed_data['station_name'] = processed_data['station_id'].map(station_names)

# transform of the temperature unit (0.1°C to °C)
processed_data['temp_mean_c'] = raw_data['TG'] / 10.0
processed_data['temp_min_c'] = raw_data['TN'] / 10.0
processed_data['temp_max_c'] = raw_data['TX'] / 10.0

# transform of the precipitation unit (0.1mm to mm)
processed_data['precip_mm'  ] = raw_data['RH'] / 10.0
# handle the special value (-1 represents <0.05mm)
processed_data.loc[raw_data['RH'] == -1, 'precip_mm'] = 0.025

# other datasets
processed_data['radiation_j_cm2'] = raw_data['Q']
processed_data['rel_humidity_pct'] = raw_data['UG']
processed_data['et0_mm'] = raw_data['EV24'] / 10.0

# calculate the water balance
processed_data['water_balance_mm'] = processed_data['precip_mm'] - processed_data['et0_mm']

# examine the data  
print("exmaine the data:")
# check the negative value
negative_values = {col: processed_data[col][processed_data[col] < 0].count() 
                  for col in processed_data.select_dtypes(include=[np.number]).columns}
print("negative value statistics:", {k: v for k, v in negative_values.items() if v > 0})

# check the missing value
missing_values = processed_data.isnull().sum()
print("missing value statistics:", missing_values[missing_values > 0] if any(missing_values > 0) else "no missing value")

# check the outliers
numeric_cols = processed_data.select_dtypes(include=[np.number]).columns
outliers = {}
for col in numeric_cols:
    mean = processed_data[col].mean()
    std = processed_data[col].std()
    outlier_count = processed_data[processed_data[col] > mean + 3*std].shape[0]
    if outlier_count > 0:
        outliers[col] = outlier_count
print("negative value statistics (>3σ):", outliers if outliers else "no obvious outliers")

# view the processed data
print("\nprocessed data preview:")
print(processed_data.head())

In [ ]:
# 1. missing value
# check the missing value
missing_rows = processed_data[processed_data.isnull().any(axis=1)]
print(f"include missing value rows: {len(missing_rows)}")
if len(missing_rows) > 0:
    print("missing value rows information:")
    print(missing_rows[['date', 'station_id', 'station_name']])

    # using linear interpolation to fill the missing value
    print("using linear interpolation to fill the missing value...")
    # group by station to interpolate
    for station_id in processed_data['station_id'].unique():
        station_mask = processed_data['station_id'] == station_id
        processed_data.loc[station_mask, 'radiation_j_cm2'] = processed_data.loc[station_mask, 'radiation_j_cm2'].interpolate(method='linear')
        processed_data.loc[station_mask, 'et0_mm'] = processed_data.loc[station_mask, 'et0_mm'].interpolate(method='linear')
        
    # recalculate the water balance
    processed_data['water_balance_mm'] = processed_data['precip_mm'] - processed_data['et0_mm']
    
    # check if there are still missing values
    remaining_missing = processed_data.isnull().sum()
    print("remaining missing value:", remaining_missing[remaining_missing > 0] if any(remaining_missing > 0) else "all filled")

# 2. abnormal value processing
# negative temperature is normal meteorological phenomenon, it is reserved.
print("\nnegative temperature is normal meteorological phenomenon, it is reserved.")
print(f"the minimum temperature range: {processed_data['temp_min_c'].min():.1f}°C  to {processed_data['temp_min_c'].max():.1f}°C")

# for extremely extreme precipitation values, we can check and visualize
extreme_precip = processed_data[processed_data['precip_mm'] > processed_data['precip_mm'].mean() + 3*processed_data['precip_mm'].std()]
print(f"\nextreme precipitation records: {len(extreme_precip)}")
if len(extreme_precip) > 0:
    print("extreme precipitation value range:", f"{extreme_precip['precip_mm'].min():.1f}mm  to {extreme_precip['precip_mm'].max():.1f}mm")
    print("time distribution:")
    print(extreme_precip['date'].dt.year.value_counts().sort_index())
    
    # check the most extreme values
    print("\nhighest precipitation records:")
    print(extreme_precip.sort_values('precip_mm', ascending=False).head(5)[['date', 'station_name', 'precip_mm']])
    
    # for extremely extreme values (e.g. >200mm/day), we can consider marking but not modifying
    extremely_high_precip = processed_data[processed_data['precip_mm'] > 200]
    if len(extremely_high_precip) > 0:
        print(f"\nextremely high precipitation (>200mm): {len(extremely_high_precip)} records")
        print(extremely_high_precip[['date', 'station_name', 'precip_mm']])
        
        # add a flag column but not change the original value
        processed_data['precip_extreme_flag'] = processed_data['precip_mm'] > 200
    else:
        processed_data['precip_extreme_flag'] = False

# recalculate the outlier statistics
print("\nprocessed outlier statistics:")
outliers = {}
for col in numeric_cols:
    if col != 'precip_extreme_flag':  # skip the flag column
        mean = processed_data[col].mean()
        std = processed_data[col].std()
        outlier_count = processed_data[processed_data[col] > mean + 3*std].shape[0]
        if outlier_count > 0:
            outliers[col] = outlier_count
print("negative value statistics (>3σ):", outliers if outliers else "no obvious outliers")

# view the processed data
print("\nprocessed data preview:")
print(processed_data.head())

## 3. Calculate Cumulative Indicators

In [ ]:
# Ensure data is sorted by date
processed_data = processed_data.sort_values(['station_id', 'date']).reset_index(drop=True)

# Calculate rolling cumulative indicators
result_data = processed_data.copy()

# Calculate separately for each station
for station_id in result_data['station_id'].unique():
    mask = result_data['station_id'] == station_id
    
    # Calculate rolling cumulative precipitation
    result_data.loc[mask, 'precip_7day_mm'] = result_data.loc[mask, 'precip_mm'].rolling(window=7, min_periods=5).sum()
    result_data.loc[mask, 'precip_30day_mm'] = result_data.loc[mask, 'precip_mm'].rolling(window=30, min_periods=25).sum()
    result_data.loc[mask, 'precip_90day_mm'] = result_data.loc[mask, 'precip_mm'].rolling(window=90, min_periods=75).sum()
    
    # Calculate rolling water balance
    result_data.loc[mask, 'water_balance_7day_mm'] = result_data.loc[mask, 'water_balance_mm'].rolling(window=7, min_periods=5).sum()
    result_data.loc[mask, 'water_balance_30day_mm'] = result_data.loc[mask, 'water_balance_mm'].rolling(window=30, min_periods=25).sum()

# View calculated data
print("\nData example after adding cumulative indicators:")
print(result_data.head())

## 4. Calculate Drought Indicators

In [ ]:
# Calculate simplified drought indicators for each station separately
final_data = result_data.copy()

for station_id in final_data['station_id'].unique():
    station_mask = final_data['station_id'] == station_id
    station_data = final_data[station_mask].copy()
    
    # Calculate historical mean and standard deviation for each calendar day
    station_data['month_day'] = station_data['date'].dt.strftime('%m-%d')
    
    # Calculate statistical values for each calendar day
    agg_data = station_data.groupby('month_day').agg({
        'precip_30day_mm': ['mean', 'std'],
        'water_balance_30day_mm': ['mean', 'std']
    })
    
    # Flatten multi-level index
    agg_data.columns = ['_'.join(col) for col in agg_data.columns]
    
    # Merge statistical values back to original data
    station_data = station_data.merge(agg_data, left_on='month_day', right_index=True)
    
    # Calculate simplified SPI and SPEI (based on Z-scores)
    station_data['simplified_spi_1month'] = np.nan
    station_data['simplified_spei_1month'] = np.nan
    
    # Only calculate standardized indices where standard deviation > 0
    valid_mask = station_data['precip_30day_mm_std'] > 0
    station_data.loc[valid_mask, 'simplified_spi_1month'] = (
        station_data.loc[valid_mask, 'precip_30day_mm'] - 
        station_data.loc[valid_mask, 'precip_30day_mm_mean']
    ) / station_data.loc[valid_mask, 'precip_30day_mm_std']
    
    valid_mask = station_data['water_balance_30day_mm_std'] > 0
    station_data.loc[valid_mask, 'simplified_spei_1month'] = (
        station_data.loc[valid_mask, 'water_balance_30day_mm'] - 
        station_data.loc[valid_mask, 'water_balance_30day_mm_mean']
    ) / station_data.loc[valid_mask, 'water_balance_30day_mm_std']
    
    # Keep only necessary columns
    keep_columns = final_data.columns.tolist() + ['simplified_spi_1month', 'simplified_spei_1month']
    station_data = station_data[keep_columns]
    
    # Update final data
    final_data.loc[station_mask] = station_data

# View final data
print("\nFinal processed data example:")
print(final_data.head())

## 5. Identify Extreme Events

In [ ]:
# Define function to identify extreme events
def identify_extreme_events(df, spi_threshold=-1.5, spei_threshold=-1.5, precip_threshold=30):
    """Identify extreme drought and flood events"""
    extreme_events = {'drought': [], 'flood': []}
    
    for station_id, station_data in df.groupby('station_id'):
        station_name = station_data['station_name'].iloc[0]
        
        # Identify drought events - consecutive 14+ days with low SPI or SPEI
        drought_mask = (station_data['simplified_spi_1month'] <= spi_threshold) | \
                      (station_data['simplified_spei_1month'] <= spei_threshold)
        
        if drought_mask.any():
            # Find drought days
            drought_days = station_data[drought_mask][['date', 'simplified_spi_1month', 'simplified_spei_1month']]
            
            # Determine start and end dates of drought events (simplified version)
            if len(drought_days) >= 14:
                start_date = drought_days['date'].min()
                end_date = drought_days['date'].max()
                duration = (end_date - start_date).days + 1
                
                # Calculate event severity
                min_spi = drought_days['simplified_spi_1month'].min()
                min_spei = drought_days['simplified_spei_1month'].min()
                severity = 'Severe' if (min_spi < -2.0 or min_spei < -2.0) else 'Moderate'
                
                extreme_events['drought'].append({
                    'station_id': station_id,
                    'station_name': station_name,
                    'start_date': start_date,
                    'end_date': end_date,
                    'duration_days': duration,
                    'min_spi': min_spi,
                    'min_spei': min_spei,
                    'severity': severity
                })
        
        # Identify flood events - daily precipitation exceeding threshold
        flood_mask = station_data['precip_mm'] >= precip_threshold
        
        if flood_mask.any():
            flood_days = station_data[flood_mask][['date', 'precip_mm']]
            
            # Simplified version: treat each day exceeding threshold as a flood event
            for _, row in flood_days.iterrows():
                extreme_events['flood'].append({
                    'station_id': station_id,
                    'station_name': station_name,
                    'date': row['date'],
                    'precipitation_mm': row['precip_mm'],
                    'severity': 'Severe' if row['precip_mm'] >= 50 else 'Moderate'
                })
    
    return extreme_events

# Identify extreme events
extreme_events = identify_extreme_events(final_data)

# View identified extreme events
print("\nIdentified drought events:")
if extreme_events['drought']:
    for event in extreme_events['drought']:
        print(f"Station: {event['station_name']}, Start: {event['start_date'].date()}, End: {event['end_date'].date()}, Duration: {event['duration_days']} days, Severity: {event['severity']}")
else:
    print("No drought events identified")
    
print("\nIdentified flood events:")
if extreme_events['flood']:
    for event in extreme_events['flood'][:5]:  # Show only first 5
        print(f"Station: {event['station_name']}, Date: {event['date'].date()}, Precipitation: {event['precipitation_mm']}mm, Severity: {event['severity']}")
    if len(extreme_events['flood']) > 5:
        print(f"... and {len(extreme_events['flood'])-5} other flood events")
else:
    print("No flood events identified")

## 6. Save Processed Data

In [ ]:
# Save processed meteorological data
output_file = os.path.join(output_dir, 'processed_weather_data.csv')
final_data.to_csv(output_file, index=False)
print(f"\nProcessed meteorological data saved to: {output_file}")

# Save extreme event information
if extreme_events['drought']:
    drought_df = pd.DataFrame(extreme_events['drought'])
    drought_file = os.path.join(output_dir, 'drought_events.csv')
    drought_df.to_csv(drought_file, index=False)
    print(f"Drought event information saved to: {drought_file}")

if extreme_events['flood']:
    flood_df = pd.DataFrame(extreme_events['flood'])
    flood_file = os.path.join(output_dir, 'flood_events.csv')
    flood_df.to_csv(flood_file, index=False)
    print(f"Flood event information saved to: {flood_file}")

## 7. Visualize Results

In [ ]:
# Plot precipitation and drought indicator time series
plt.figure(figsize=(12, 8))

# Plot separate graphs for each station
for station_id, station_data in final_data.groupby('station_id'):
    plt.subplot(2, 1, 1 if station_id == 275 else 2)
    
    # Plot monthly precipitation
    plt.plot(station_data['date'], station_data['precip_30day_mm'], 'b-', label='30-day cumulative precipitation (mm)')
    
    # Plot drought indicators
    plt.plot(station_data['date'], station_data['simplified_spi_1month'], 'r-', label='Simplified SPI (1-month)')
    
    # Add drought and wet threshold lines
    plt.axhline(y=-1.5, color='r', linestyle='--', alpha=0.7)
    plt.axhline(y=1.5, color='b', linestyle='--', alpha=0.7)
    
    plt.title(f"{station_data['station_name'].iloc[0]} Station Precipitation and Drought Indicators")
    plt.ylabel('Precipitation (mm) / SPI Value')
    plt.grid(True, alpha=0.3)
    plt.legend()

plt.tight_layout()
fig_file = os.path.join(output_dir, 'precipitation_drought_timeseries.png')
plt.savefig(fig_file, dpi=300)
plt.close()
print(f"\nPrecipitation and drought indicator time series plot saved to: {fig_file}")